In [ ]:
import pandas as pd
data = pd.read_csv("amazon-purchases.csv")

#type(data["Order Date"][0]) #Date Column is a string
#Create a list with prime days inside it
prime_days = ["2022-07-12", "2022-07-13", "2021-06-21", "2021-06-22", "2020-10-13", "2020-10-14", "2019-07-15", "2019-07-16", "2018-07-17", "2018-07-18",]

prime_purchase = []
for i in data["Order Date"]:
    if i in prime_days:
        prime_purchase.append(1)
    else:
        prime_purchase.append(0)
data["Prime Purchase"] = prime_purchase

data["Prime Purchase"].value_counts()

In [ ]:
departments = ['Clothing, Shoes & Jewelry',
               'Everything Else',
               'Home & Kitchen',
               'Sports & Outdoors',
               'Tools & Home Improvement',
               'Electronics',
               'Apps & Games',
               'Health & Personal Care',
               'Office Products',
               'Industrial & Scientific',
               'Grocery & Gourmet Food',
               'Baby',
               'Pet Supplies',
               'Automotive Parts and Accessories',
               'Beauty & Personal Care',
               'Cell Phones & Accessories',
               'Garden & Outdoor',
               'Video Games',
               'Books',
               'Kindle Accessories',
               'Appliances',
               'Arts, Crafts & Sewing',
               'Gift Cards',
               'Musical Instruments',
               'Toys & Games',
               'Alexa Skills',
               'CDs & Vinyl',
               'Software',
               'Movies & TV']

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(data['Title'][0])

In [ ]:
doc

In [ ]:
textcat = nlp.add_pipe("textcat_multilabel")

In [ ]:
for i in departments:
    textcat.add_label(i)

In [ ]:
textcat.label_data

In [ ]:
# Assuming you have training data in the form of (text, labels)
train_data = [
    ("Simple Joys by Carter's Baby 3-Pack Neutral Cotton Sleeper Gown", "Clothing, Shoes & Jewelry"),
    ("Amazon Essentials Men's Fleece Open Bottom Sweatpant (Available in Big & Tall)", "Clothing, Shoes & Jewelry"),
    ("Amazon Essentials Men's Performance Comfortable Cotton Cushioned Breathable Athletic Crew Sports Socks, 10 Pairs", "Clothing, Shoes & Jewelry"),
    ("Amazon Essentials Women's Cotton Bikini Brief Underwear (Available in Plus Size), Pack of 6", "Clothing, Shoes & Jewelry"),
    ("Bellivera Women's Stand Collar Lightweight Quilted Puffer Jacket Padded Zip Coat for Fall and Winter", "Clothing, Shoes & Jewelry"),
    ("Amazon Basics 2-Ply Flex-Sheets Paper Towels, 12 Basics Rolls = 32 Regular Rolls, Everyday Value with 150 Sheets per Roll","Home & Kitchen"),
    ("Amazon Basics Flextra Tall Kitchen Drawstring Trash Bags, Unscented, 13 Gallon, 120 Count, Pack of 1","Home & Kitchen"),
    ("Amazon Basics Microfiber Cleaning Cloths, Non-Abrasive, Reusable and Washable, Pack of 24, Blue/White/Yellow, 16 x 12","Home & Kitchen"),
    ("Amazon Basics Reversible Lightweight Microfiber Comforter Blanket, Twin/Twin XL, Navy/Sky Blue","Home & Kitchen"),
    ("Amazon Basics Stainless Steel Wire Whisk Set - 3-Piece", {"cats": {"Home & Kitchen": 1}}),
]

from spacy.tokens import DocBin
from spacy.training import Example

nlp = spacy.blank("en")
training_data = [
  ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")]),
]
# the DocBin will store the example documents
db = DocBin()
for text, annotations in training_data:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    doc.ents = ents
# Initialize the textcat model with the correct number of labels
textcat = nlp.add_pipe("textcat_multilabel", config={"exclusive_classes": True, "architecture": "bow"})

for label in departments:
    textcat.add_label(label)

# Train the textcat model
optimizer = nlp.create_optimizer()
for i in range(10):  # Number of training iterations
    losses = {}
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer, losses=losses)
    print(losses)

# Now you can use the textcat model for predictions
textcat(doc)